In [1]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

C:\Users\qchasserieau\AppData\Local\Continuum\anaconda3\envs\quetzal_env\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [1]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
from quetzal.io import excel
from syspy.spatial.spatial import nearest_geometry
import numpy as np
import pandas as pd
import geopandas as gpd

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [3]:
sm = stepmodel.read_zippedpickles(r'../../model/{scen}/assignment'.format(scen=scenario))

zone_to_transit: 100%|█████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.65it/s]


In [4]:
counts = gpd.read_file(r'../../inputs/vehicle_counts.geojson')
screenlines = gpd.read_file(r'../../inputs/screenlines.geojson')
from shapely import geometry
screenlines['geometry']  = screenlines['geometry'].apply(lambda g: g[0])
counts['geometry']  = counts['geometry'].apply(lambda g: g[0])
counts['left_nesw'] = counts['left_nesw'].apply(str.upper)
counts['right_nesw'] = counts['right_nesw'].apply(str.upper)

In [5]:
counts

,name,left_to,right_to,screenline,left_nesw,right_nesw,geometry
0,D3,dil chora,dechatu,D,W,E,"LINESTRING (814121.960 1061005.653, 814095.570..."
1,F5,connel,mesgid taliyan,F,W,E,"LINESTRING (815048.912 1061164.704, 815038.967..."
2,D4,seido,kefira,D,W,E,"LINESTRING (814448.283 1060501.240, 814367.333..."
3,F4,chat tera,legehare,F,W,E,"LINESTRING (815384.493 1061647.030, 815354.197..."
4,F3,kelafo,bilal,F,W,E,"LINESTRING (815371.497 1061847.316, 815350.445..."
5,F1,taiwan,ashewa,F,W,E,"LINESTRING (814886.927 1062076.375, 814826.017..."
6,F2,taiwan,ashewa,F,W,E,"LINESTRING (814960.390 1062028.935, 814900.424..."
7,B2,melka,seido,B,W,E,"LINESTRING (810621.081 1062357.904, 810663.270..."
8,B1,melka,seido,B,W,E,"LINESTRING (810731.065 1062840.854, 810759.759..."
9,A1,melka,seido,A,W,E,"LINESTRING (805611.273 1064025.198, 805626.805..."


In [340]:
def get_angle(g):
    c = np.array(g.coords)
    seg = c[-1] - c[0]
    return np.angle(complex(*seg), deg=True)

def a_crosses_b_to_the_right_or_left(a, b):
    if (get_angle(a) - get_angle(b)) % 360 < 180:
        return 'right'
    else :
        return 'left'

## COUNTS

In [341]:
flows = ['pt', 'walk', 'car']

one = counts.drop(['id'], axis=1, errors='ignore').set_index('name')
distance = nearest_geometry(one, sm.road_links, n_neighbors=10, n_neighbors_centroid=100)
intersections = distance.loc[distance['actual_distance'] == 0]

intersections['direction'] = intersections.apply(
    lambda i: a_crosses_b_to_the_right_or_left(
        a=sm.road_links.loc[i['ix_many'], 'geometry'],
        b=one.loc[i['ix_one'], 'geometry']
    ), axis=1
)

hinge = pd.merge(
    intersections[['ix_one',  'ix_many', 'direction']].rename(
        columns={ 'ix_many': 'road_link'}),
    one.drop(['geometry'], axis=1),
    left_on='ix_one',
    right_index=True
)
flow_counts = pd.merge(
    hinge, sm.road_links[flows], 
    left_on='road_link', right_index=True
).fillna(0).set_index('ix_one')

nearest_link: 100%|█████████████████████████████████████████████████████████████| 2700/2700 [00:00<00:00, 99940.17it/s]


In [342]:
# SCREENLINES
links = sm.road_links.loc[sm.road_links[flows].T.sum() > 0]
hinge_values = []
for ix, screengeo in screenlines[['name', 'geometry']].values:
    for linkix, linkgeo in links.reset_index()[['index', 'geometry']].values:
        if linkgeo.distance(screengeo) == 0:
            direction = a_crosses_b_to_the_right_or_left(linkgeo, screengeo)
            hinge_values.append([ix, linkix, direction])
            
hinge = pd.DataFrame(hinge_values, columns=['screenline', 'road_link', 'direction'])
flow_screelines = pd.merge(hinge, sm.road_links[flows], left_on='road_link', right_index=True).fillna(0)

In [356]:
flow_screelines.head()

,screenline,road_link,direction,pt,walk,car
0,B,road_link_144,right,1767.619336,209.041868,517.312621
1,B,road_link_1991,right,0.000000,9.945601,4.517319
2,B,road_link_5089,left,3283.182534,467.603811,904.440924
3,B,road_link_7606,left,0.000000,112.353803,239.261430
4,B,road_link_9834,left,0.000000,14.140994,2.644377


In [357]:
flow_counts.head()

,road_link,direction,left_to,right_to,screenline,left_nesw,right_nesw,pt,walk,car
ix_one,,,,,,,,,,
A1,road_link_3912,right,melka,seido,A,W,E,1403.314381,75.667288,320.689670
A1,road_link_11718,left,melka,seido,A,W,E,2672.330485,174.494134,686.470601
B1,road_link_5089,left,melka,seido,B,W,E,3283.182534,467.603811,904.440924
B1,road_link_144,right,melka,seido,B,W,E,1767.619336,209.041868,517.312621
B2,road_link_9834,left,melka,seido,B,W,E,0.000000,14.140994,2.644377


In [358]:
cg = np.round(
    flow_counts.groupby(['screenline', 'direction'])[flows].sum(),
    -1
).astype(int)

In [359]:
sg = np.round(
    flow_screelines.groupby(['screenline', 'direction'])[flows].sum(),
    -1
).astype(int)

In [360]:
sg.T

screenline     A           B           C           D           E           F  \
direction   left right  left right  left right  left right  left right  left   
pt          2670  1400  3280  1770  7670  9470  6770  9930  9830  6200  1780   
walk         170    80   590   330  4350  5640  6770  8770  7450  4850  2790   
car          690   320  1150   800  2500  3310  2470  3280  2980  1740   750   

screenline        
direction  right  
pt          4360  
walk        4960  
car         1110

In [361]:
cg.T

screenline     A           B           C           D           E           F  \
direction   left right  left right  left right  left right  left right  left   
pt          2670  1400  3280  1770  7280  9470  6770  9930  9760  5910  1780   
walk         170    80   480   220  2780  4790  5920  7150  6780  4120  2500   
car          690   320   910   520  1910  2890  1910  2510  2880  1530   670   

screenline        
direction  right  
pt          4360  
walk        4710  
car         1060

In [362]:
(cg - sg).T

screenline    A          B           C          D          E          F      
direction  left right left right  left right left right left right left right
pt            0     0    0     0  -390     0    0     0  -70  -290    0     0
walk          0     0 -110  -110 -1570  -850 -850 -1620 -670  -730 -290  -250
car           0     0 -240  -280  -590  -420 -560  -770 -100  -210  -80   -50

### COUNTS TREATMENT 

In [2]:
clean_counts = pd.read_excel(r'../../inputs/clean_counts.xlsx')

In [ ]:
counts_hpm = clean_counts.loc[(clean_counts['hour'] >= 7) &
                                   (clean_counts['hour']<8)]

In [ ]:
capacity_bus = 40
capacity_minibus = 20

bus = pd.DataFrame({
    'position' : counts_hpm['position'],
    'direction' : counts_hpm['direction'],
    'volume' : counts_hpm['bus']*capacity_bus/100,
    'mode_survey' : 'bus',
    'mode_model' : 'pt'
}).dropna()

minibus = pd.DataFrame({
    'position' : counts_hpm['position'],
    'direction' : counts_hpm['direction'],
    'volume' : counts_hpm['minibus']*capacity_minibus/100,
    'mode_survey' : 'minibus',
    'mode_model' : 'pt'
}).dropna()

car = pd.DataFrame({
    'position' : counts_hpm['position'],
    'direction' : counts_hpm['direction'],
    'volume' : counts_hpm['car'],
    'mode_survey' : 'car',
    'mode_model' : 'car'
}).dropna()

moto = pd.DataFrame({
    'position' : counts_hpm['position'],
    'direction' : counts_hpm['direction'],
    'volume' : counts_hpm['moto'],
    'mode_survey' : 'moto',
    'mode_model' : 'car'
}).dropna()

bajaj = pd.DataFrame({
    'position' : counts_hpm['position'],
    'direction' : counts_hpm['direction'],
    'volume' : counts_hpm['bajaj'],
    'mode_survey' : 'bajaj',
    'mode_model' : 'pt',
    #'origin' : counts_hpm['bajaj_origin'],
    #'destination' : counts_hpm['bajaj_destination'],
}).dropna(subset=['volume'])

In [ ]:
counts_all=pd.concat([bus,minibus,car,moto,bajaj]).reset_index(drop=True)

In [ ]:
flow_counts['direction_id'] = flow_counts['direction']
for i in range(len(flow_counts)):
    if flow_counts['direction'][i]=='right':
        flow_counts['direction_id'][i]=flow_counts['right_to'][i]
    else :
        flow_counts['direction_id'][i]=flow_counts['left_to'][i]

In [ ]:
counts_position=counts_all.groupby(['position', 'direction','mode_survey']).sum(['volume']).unstack('mode_survey').reset_index()

In [ ]:
counts_position

In [ ]:
flow_model = flow_counts.groupby(['ix_one', 'direction_id','direction', 'screenline', 'left_nesw', 'right_nesw'])[['pt', 'car', 'walk']].sum()

flow_model.reset_index(inplace=True)

flow_model.rename(columns={'direction':'direction_lr'}, inplace=True)

flow_model.head()

In [ ]:
counts_comp = pd.merge(counts_position, flow_model, left_on=['position', 'direction'], right_on=['ix_one', 'direction_id'])

counts_comp.rename(columns={('position',''):'position',
                            ('direction',''):'direction',
                            ('volume','bajaj'):'bajaj',
                            ('volume','bus'):'bus',
                            ('volume','car'):'car_counts',
                            ('volume','minibus'):'minibus',
                            ('volume','moto'):'moto',
                           }, inplace=True)

counts_comp.fillna(0, inplace=True)

counts_comp['modeled']=counts_comp['pt']+counts_comp['car']
counts_comp['counted']=counts_comp['bajaj']+counts_comp['bus']+counts_comp['car_counts']+counts_comp['minibus']+counts_comp['moto']

counts_comp['GEH'] = pow(2*pow(counts_comp['modeled']-counts_comp['counted'],2)/(counts_comp['modeled']+counts_comp['counted']),0.5)

counts_comp['deviation'] = (counts_comp['modeled']-counts_comp['counted'])/counts_comp['counted']

counts_comp.drop(columns=['ix_one','direction_id'], inplace=True)

In [ ]:
counts_comp.head()

In [ ]:
df = pd.DataFrame({'x':[-100,6000], 'y':[-100,6000]})
ax1 = df.plot(kind='line', x='x', y='y', legend=False, color='lightskyblue')
counts_plot = counts_comp.plot(kind='scatter', x='counted', y='modeled', xlim=[-100,6000], ylim=[-100,6000], figsize=[5,5], ax=ax1)

In [ ]:
count_geh_plot = counts_comp['GEH'].plot(kind='hist')

In [ ]:
screenlines_comp = counts_comp[['screenline', 'direction_lr','modeled','counted']].groupby(['screenline', 'direction_lr']).sum(['modeled','counted'])
screenlines_comp['GEH'] = pow(2*pow(screenlines_comp['modeled']-screenlines_comp['counted'],2)/(screenlines_comp['modeled']+screenlines_comp['counted']),0.5)
screenlines_comp['deviation'] = (screenlines_comp['modeled']-screenlines_comp['counted'])/screenlines_comp['counted']

In [ ]:
screenlines_comp

In [ ]:
screenlines_comp.sum()

In [ ]:
df = pd.DataFrame({'x':[-100,10000], 'y':[-100,10000]})
ax2 = df.plot(kind='line', x='x', y='y', legend=False, color='lightskyblue')
screenlines_plot = screenlines_comp.plot(kind='scatter', x='counted', y='modeled', xlim=[-100,10000], ylim=[-100,10000], figsize=[5,5], ax=ax2)

## Export 

In [ ]:
counts_comparison = counts_plot.get_figure()
counts_comparison.savefig(r'../../model/{scen}/calibration/counts_{scen}.png'.format(scen=scenario), bbox_inches='tight')

In [ ]:
GEH_comparison = count_geh_plot.get_figure()
GEH_comparison.savefig(r'../../model/{scen}/calibration/GEH_histo_{scen}.png'.format(scen=scenario), bbox_inches='tight')

In [ ]:
screenlines_comparison = screenlines_plot.get_figure()
screenlines_comparison.savefig(r'../../model/{scen}/calibration/screenlines_{scen}.png'.format(scen=scenario), bbox_inches='tight')

In [ ]:
writer = pd.ExcelWriter(r'../../model/{scen}/calibration/calibration_{scen}.xlsx'.format(scen=scenario), mode='a')

assign_ind = pd.DataFrame({'Average GEH (counts)' : counts_comp['GEH'].mean(),
                          'Average deviation (counts)' : counts_comp['deviation'].mean(),
                          'Average GEH (screenlines)' : screenlines_comp['GEH'].mean(),
                          'Average deviation (screenlines)' : screenlines_comp['deviation'].mean()
                          }, index=range(1))

assign_ind.T.to_excel(writer, sheet_name='assignment')
writer.save()

# END

In [ ]:
end_of_notebook